# Simple Family Tree with RocksDB

A basic example showing how to store and retrieve family data using RocksDB.

**Family Structure:**
- John (Father) + Alice (Mother)
- Children: David (Male, Married) + Joe (Male, Single)
- David + Sarah (Wife) → Michael (Son)

## Step 1: Install and Import

In [1]:
# Check virtual environment status
import sys
import os

print("=== Virtual Environment Status ===")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Virtual environment active: {hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)}")

if '.venv' in sys.executable:
    print("✓ Running in virtual environment (.venv)")
else:
    print("⚠️  Not running in expected virtual environment")

print(f"Current working directory: {os.getcwd()}")
print("=" * 50)

=== Virtual Environment Status ===
Python executable: /workspaces/curiosity/.venv/bin/python
Python version: 3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]
Virtual environment active: True
✓ Running in virtual environment (.venv)
Current working directory: /workspaces/curiosity


In [2]:
# Install rocksdict (easier alternative to python-rocksdb)
import subprocess
import sys

def install_rocksdict():
    """Install rocksdict which is easier than python-rocksdb"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "rocksdict"])
        print("✓ rocksdict installed successfully!")
        return True
    except subprocess.CalledProcessError:
        print("❌ Failed to install rocksdict")
        return False

try:
    import rocksdict
    print("✓ rocksdict already installed")
except ImportError:
    print("Installing rocksdict...")
    success = install_rocksdict()
    if not success:
        print("❌ Failed to install rocksdict")
        print("Please run manually: pip install rocksdict")
    else:
        import rocksdict
        print("✓ rocksdict imported successfully")

Installing rocksdict...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 3.1 MB/s eta 0:00:00a 0:00:01
✓ rocksdict installed successfully!
✓ rocksdict imported successfully


In [3]:
# Import libraries
import json
import rocksdict

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 2: Setup Database

In [4]:
# Create RocksDB database using rocksdict
from rocksdict import Rdict, Options

# Create database with options
opts = Options()
opts.create_if_missing(True)
db = Rdict("family_db", options=opts)

print("RocksDB database created successfully!")

RocksDB database created successfully!


## Step 3: Helper Functions

In [5]:
def add_person(name, sex, parent=None, spouse=None):
    """Add a person to the family tree"""
    person = {
        'name': name,
        'sex': sex,
        'parent': parent,
        'spouse': spouse
    }
    # Store in RocksDB (rocksdict handles encoding automatically)
    db[name] = json.dumps(person)
    print(f"Added: {name} ({sex})")

def get_person(name):
    """Get a person from the database"""
    try:
        data = db.get(name)
        if data:
            return json.loads(data)
    except:
        pass
    return None

def show_person(name):
    """Display person details"""
    person = get_person(name)
    if person:
        print(f"Name: {person['name']}")
        print(f"Sex: {person['sex']}")
        print(f"Parent: {person['parent'] or 'None'}")
        print(f"Spouse: {person['spouse'] or 'None'}")
        print("-" * 20)
    else:
        print(f"{name} not found")

def show_all():
    """Show all people in the database"""
    print("=== Family Tree ===")
    # Iterate through all keys in RocksDB
    for key in db.keys():
        name = key
        show_person(name)

print("Helper functions defined!")

Helper functions defined!


## Step 4: Create Family

In [6]:
# Add parents
add_person("John", "Male")
add_person("Alice", "Female", spouse="John")

# Update John's spouse
john = get_person("John")
john['spouse'] = "Alice"
db["John"] = json.dumps(john)

print("\nParents added!")

Added: John (Male)
Added: Alice (Female)

Parents added!


In [7]:
# Add children of John and Alice
add_person("David", "Male", parent="John")
add_person("Joe", "Male", parent="John")

print("\nChildren added!")

Added: David (Male)
Added: Joe (Male)

Children added!


In [8]:
# Add David's wife
add_person("Sarah", "Female", spouse="David")

# Update David's spouse
david = get_person("David")
david['spouse'] = "Sarah"
db["David"] = json.dumps(david)

# Add David and Sarah's child
add_person("Michael", "Male", parent="David")

print("\nDavid's family added!")

Added: Sarah (Female)
Added: Michael (Male)

David's family added!


## Step 5: View Family Tree

In [9]:
# Show complete family tree
show_all()

=== Family Tree ===
Name: Alice
Sex: Female
Parent: None
Spouse: John
--------------------
Name: David
Sex: Male
Parent: John
Spouse: Sarah
--------------------
Name: Joe
Sex: Male
Parent: John
Spouse: None
--------------------
Name: John
Sex: Male
Parent: None
Spouse: Alice
--------------------
Name: Michael
Sex: Male
Parent: David
Spouse: None
--------------------
Name: Sarah
Sex: Female
Parent: None
Spouse: David
--------------------


## Step 6: Simple Queries

In [10]:
def find_males():
    """Find all male family members"""
    males = []
    # Iterate through RocksDB
    for name in db.keys():
        person = get_person(name)
        if person and person['sex'] == 'Male':
            males.append(person['name'])
    return males

def find_children(parent_name):
    """Find children of a specific parent"""
    children = []
    # Iterate through RocksDB
    for name in db.keys():
        person = get_person(name)
        if person and person['parent'] == parent_name:
            children.append(person['name'])
    return children

# Examples
print("All males:", find_males())
print("John's children:", find_children("John"))
print("David's children:", find_children("David"))

All males: ['David', 'Joe', 'John', 'Michael']
John's children: ['David', 'Joe']
David's children: ['Michael']


In [11]:
# Look up specific people
print("=== Individual Details ===")
show_person("David")
show_person("Joe")
show_person("Michael")

=== Individual Details ===
Name: David
Sex: Male
Parent: John
Spouse: Sarah
--------------------
Name: Joe
Sex: Male
Parent: John
Spouse: None
--------------------
Name: Michael
Sex: Male
Parent: David
Spouse: None
--------------------


## Step 7: Raw Data

In [12]:
# See all data stored in RocksDB
print("All family data in RocksDB:")
for name, data in db.items():
    person_data = json.loads(data)
    print(f"{name}: {person_data}")

All family data in RocksDB:
Alice: {'name': 'Alice', 'sex': 'Female', 'parent': None, 'spouse': 'John'}
David: {'name': 'David', 'sex': 'Male', 'parent': 'John', 'spouse': 'Sarah'}
Joe: {'name': 'Joe', 'sex': 'Male', 'parent': 'John', 'spouse': None}
John: {'name': 'John', 'sex': 'Male', 'parent': None, 'spouse': 'Alice'}
Michael: {'name': 'Michael', 'sex': 'Male', 'parent': 'David', 'spouse': None}
Sarah: {'name': 'Sarah', 'sex': 'Female', 'parent': None, 'spouse': 'David'}


## Summary

This simple notebook shows:
1. How to install and use RocksDB with Python
2. Basic read/write operations 
3. A simple family tree with the exact structure you requested:
   - John (father) + Alice (mother)
   - David (son, married to Sarah with child Michael)
   - Joe (son, unmarried)
4. Simple queries to find specific family members

The database file is saved locally and persists between runs.

In [13]:
# Close the database
db.close()
print("Database closed.")

Database closed.
